In [1]:
import cv2
import os
from ultralytics import YOLO

In [2]:
model_weights_path = '../weights/yolov10n.pt' # Путь до весов
video_path = '../test_videos/road.mp4' # Путь до видео
crops_dir = '../dataset' # Путь до формируемого датасета

In [3]:
# Инициализируем модель YOLOv8
model = YOLO(model_weights_path)

# Создаем общую папку для обрезков, если ее нет
os.makedirs(crops_dir, exist_ok=True)

In [4]:
# Открываем видеофайл
cap = cv2.VideoCapture(video_path)

frame_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break  # Выходим из цикла, если кадры закончились

    frame_count += 1
    print(f"Обработка кадра {frame_count}")

    # Обнаружение объектов на текущем кадре
    results = model(frame)

    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy()  # Координаты боксов
        scores = result.boxes.conf.cpu().numpy()  # Доверительные оценки
        labels = result.boxes.cls.cpu().numpy().astype(int)  # Метки классов

        for box, score, label in zip(boxes, scores, labels):
            x1, y1, x2, y2 = map(int, box)
            cropped_object = frame[y1:y2, x1:x2]

            label_name = model.names[label]  # Получаем имя класса
            label_dir = os.path.join(crops_dir, label_name)
            os.makedirs(label_dir, exist_ok=True)

            # Сохраняем обрезанный объект
            crop_filename = f"{label_name}_{frame_count}_{score:.2f}.jpg"
            crop_path = os.path.join(label_dir, crop_filename)
            cv2.imwrite(crop_path, cropped_object)

            # Рисуем B-боксы на кадре
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Зеленый цвет, толщина 2
            cv2.putText(frame, f"{label_name} {score:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 
                        0.5, (0, 255, 0), 2)  # Отображаем метку и доверие над боксом

    # Отображаем кадр с боксов
    cv2.imshow('Frame with BBoxes', frame)

    # Нажмите 'q' для выхода
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Обработка кадра 1

0: 384x640 11 cars, 1 truck, 104.0ms
Speed: 3.9ms preprocess, 104.0ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)
Обработка кадра 2

0: 384x640 11 cars, 1 truck, 87.1ms
Speed: 10.7ms preprocess, 87.1ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
Обработка кадра 3

0: 384x640 11 cars, 2 trucks, 103.1ms
Speed: 3.5ms preprocess, 103.1ms inference, 0.1ms postprocess per image at shape (1, 3, 384, 640)
Обработка кадра 4

0: 384x640 11 cars, 2 trucks, 114.1ms
Speed: 1.8ms preprocess, 114.1ms inference, 0.1ms postprocess per image at shape (1, 3, 384, 640)
Обработка кадра 5

0: 384x640 11 cars, 2 trucks, 87.6ms
Speed: 1.6ms preprocess, 87.6ms inference, 0.1ms postprocess per image at shape (1, 3, 384, 640)
Обработка кадра 6

0: 384x640 10 cars, 1 truck, 69.6ms
Speed: 1.1ms preprocess, 69.6ms inference, 0.1ms postprocess per image at shape (1, 3, 384, 640)
Обработка кадра 7

0: 384x640 7 cars, 2 trucks, 61.8ms
Speed: 1.6ms preprocess